# Reproduction of the winner algorithm for ISMIR 2004

This notebook tries to reproduce the approach of the winner algorithm for ISMIR 2004, here are the main sections:

1. [Importing modules and some constants](#imports)
2. [Loading the dataset](#loading_dataset)
3. [Processing files](#processing_files)
4. [Evaluating the accuracy](#evaluation)
5. [Showing the results](#showing_results)

<br>

If you want so see in detail how each file is processed you can take a look at this [Single file analysis notebook](single_file_analysis.ipynb)

#### Importing modules and some constants
<a id="imports"></a>

In [1]:
import sys
sys.path.append('../')

import time
from IPython.display import clear_output
from utils.constants import *
from utils.eval_func import *
from utils.process import *
from utils.utils import *
from utils.data_management import *

#### Loading the dataset
<a id="loading_dataset"></a>

In [2]:
files = load_dataset() #list with the file names without the extension

#### Processing files
<a id="processing_files"></a>

The files are processed and the result is saved into csv files

In [3]:
times = []
for i,filename in enumerate(files):
    print("{0}/{1}".format(i+1,len(files))) #progress
    
    audio = load_file(filename) #loads the audio and applies an equal loudness filter
    
    start = time.time()
    pitch = process_audio(audio) #returns the predominant pitch per frame
    times.append(time.time()-start)
    
    save_pitch(pitch,filename) #saves the pitches to the csv
    clear_output()

print("In average, each file took:",sum(times)/len(times), " s to compute")

1/20


KeyboardInterrupt: 

#### Evaluating the accuracy
<a id="evaluation"></a>

In [ ]:
if os.path.exists(ERROR_CSV_FILE): os.remove(ERROR_CSV_FILE)

for filename in files:
    compute_error(filename)

calculate_average_tsv(ERROR_CSV_FILE)
calculate_difference_with_reference(ERROR_CSV_FILE, ERROR_WINNER_CSV_FILE)